In [ ]:
#import modules 
!pip install selenium
!pip install textmagic
!apt-get update 
!apt install chromium-chromedriver
from textmagic.rest import TextmagicRestClient
from selenium import webdriver
import time
from bs4 import BeautifulSoup
import pandas as pd
from selenium.webdriver.common.keys import Keys 
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC 
#initialize webdriver  
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
driver =webdriver.Chrome('chromedriver',chrome_options=chrome_options)
# BaseURL of Yahoo Finance website.
URL = "https://finance.yahoo.com/"
# List of companies, we're scanning.
Companies = ["BILI", "Square", "Pfizer", "Ali Baba", "Roku", "Palantir", "Boeing"]
for Company in Companies:
  #navigate to web page
  driver.get(URL)
  time.sleep(2)
  # Enter name of company in searchbox, and wait for 2 seconds.
  driver.find_element_by_xpath("//input[@placeholder = 'Search for news, symbols or companies']").send_keys(Company)
  time.sleep(2)
  # Click on Search icon and wait for 2 seconds.
  driver.find_element_by_xpath("//button[@id= 'header-desktop-search-button']").click()
  time.sleep(2)
  # Driver clicks on Historical Data tab and sleeps for 2 seconds.
  driver.find_element_by_xpath("//span[text() = 'Historical Data']").click()
  time.sleep(2)
  # Driver scrolls down three times to load the table.
  for i in range(0,3):
    driver.execute_script("window.scrollBy(0,5000)")
  time.sleep(2)
  # Fetch the webpage and store in a variable.
  webpage = driver.page_source
  # print the fetched webpage.
  #print(webpage)
  # Web page fetched from driver is parsed using Beautiful Soup.
  HTMLPage = BeautifulSoup(driver.page_source, 'html.parser')
  currentprice = HTMLPage.find_all('div', {'class':'My(6px) Pos(r) smartphone_Mt(6px)'})[0].find('span').text
  # Table is searched using class and stored in another variable.
  Table = HTMLPage.find('table', class_='W(100%) M(0)')
  # List of all the rows is store in a variable 'Rows'.
  Rows = Table.find_all('tr', class_='BdT Bdc($seperatorColor) Ta(end) Fz(s) Whs(nw)')
  # Empty list is created to store the data
  historical_data = []
  # Loop to go through each row of table
  for i in range(0, len(Rows)):
    try:
      # Empty dictionary to store data present in each row
      RowDict = {}
      # Extracted all the columns of a row and stored in a variable
      Values = Rows[i].find_all('td')
      
      # Values (Open, High, Close etc.) are extracted and stored in dictionary
      if len(Values) == 7:
        RowDict["Date"] = Values[0].find('span').text.replace(',', '')
        RowDict["Open"] = Values[1].find('span').text.replace(',', '')
        RowDict["High"] = Values[2].find('span').text.replace(',', '')
        RowDict["Low"] = Values[3].find('span').text.replace(',', '')
        RowDict["Close"] = Values[4].find('span').text.replace(',', '')
        RowDict["Adj Close"] = Values[5].find('span').text.replace(',', '')
        RowDict["Volume"] = Values[6].find('span').text.replace(',', '')
      # Uncomment below print statement if required
        # print(RowDict) 
        # Dictionary is appended in list
        historical_data.append(RowDict)
    except:
      # To check the exception caused for which company
        print("Row Number: " + str(i))
    finally:
      # To move to the next row
      i = i + 1
  # Converted list of dictionaries to a Dataframe.
  historical_data = pd.DataFrame(historical_data)
  support = min(historical_data.iat[1,3], historical_data.iat[2,3],historical_data.iat[3,3],historical_data.iat[4,3],historical_data.iat[5,3])
  resistance = max(historical_data.iat[1,2], historical_data.iat[2,2],historical_data.iat[3,2],historical_data.iat[4,2],historical_data.iat[5,2])
  print(support)
  print(resistance)
  #note TextMagic account is private, input your own Username and Token to complete SMS message
  if (currentprice > resistance or currentprice < support):
    print(Company + " has broken it's 5 day support or resistance. Alert should be sent for this company to our user.")
    username = "cchandel2002"
    token = "*************"
    client = TextmagicRestClient(username, token)
    message = client.messages.create(phones="2267004807", text="5 day support or resistance has been broken for " + Company)
  else:
    print(Company + " has not broken it's 5day support or resistance. No alert should be sent for this company to our user.")